In [1]:
with open("program_file_reading.py") as file:
    raw_code = file.read()

print(raw_code)

if __name__ == '__main__':
    # with open("main.py") as file:
    #     raw_code = file.read()
    # print(raw_code)

    import pylint.lint

    # pylint_opts = ['--disable=line-too-long', 'main.py']
    # pylint_opts = ['--disable=line-too-long', 'program_file_reading.py']
    pylint_opts = ['--disable=line-too-long', '/home/asif/PycharmProjects/ScrappingScript/Scrapping_Time.py']
    try:
        pylint.lint.Run(pylint_opts)
    except Exception:
        print(Exception)



In [4]:
import pylint.lint
from pylint import epylint as lint

module_name = 'test.py'

pylint_opts = ['--disable=line-too-long', module_name]

# try:
#     # pylint.lint.Run(pylint_opts)
#     (pylint_stdout, pylint_stderr) = lint.py_run('test.py', return_std=True)
# except Exception:
#     print(Exception)

(pylint_stdout, pylint_stderr) = lint.py_run(module_name, return_std=True)


In [6]:
# pylint_stdout.read()
pylint_stdout.getvalue()

str

In [4]:
pylint_stderr.read()

''

In [6]:
from io import StringIO

from pylint.lint import Run
from pylint.reporters.text import TextReporter

pylint_output = StringIO()  # Custom open stream
reporter = TextReporter(pylint_output)
Run(["test_file.py"], reporter=reporter, do_exit=False)
print(pylint_output.getvalue())  # Retrieve and print the text report



************* Module test_file.py
test_file.py:1:0: F0001: No module named test_file.py (fatal)



In [24]:
%tb

SystemExit: 20

In [7]:
import astroid
from astroid import nodes

from pylint.checkers import BaseChecker
from pylint.interfaces import IAstroidChecker
from pylint.lint import PyLinter

from typing import TYPE_CHECKING

if TYPE_CHECKING:
    from pylint.lint import PyLinter


def register(linter: "PyLinter") -> None:
    """This required method auto registers the checker during initialization.

    :param linter: The linter to register the checker to.
    """
    linter.register_checker(UniqueReturnChecker(linter))


class UniqueReturnChecker(BaseChecker):
    __implements__ = IAstroidChecker

    name = 'unique-returns'
    priority = -1
    msgs = {
        'W0001': (
            'Returns a non-unique constant.',
            'non-unique-returns',
            'All constants returned in a function should be unique.'
        ),
    }

    options = (
        (
            'ignore-ints',
            {
                'default': False, 'type': 'yn', 'metavar': '<y or n>',
                'help': 'Allow returning non-unique integers',
            }
        ),
    )

    def __init__(self, linter: PyLinter = None) -> None:
        super(UniqueReturnChecker, self).__init__(linter)
        # super().__init__(linter)
        self._function_stack = []

    def visit_functiondef(self, node: nodes.FunctionDef) -> None:
        self._function_stack.append([])

    def leave_functiondef(self, node: nodes.FunctionDef) -> None:
        self._function_stack.pop()

    def visit_return(self, node: nodes.Return) -> None:
        if not isinstance(node.value, nodes.Const):
            return

        for other_return in self._function_stack[-1]:
            if (node.value.value == other_return.value.value and
                    not (self.config.ignore_ints and node.value.pytype() == int)):
                self.add_message(
                    'non-unique-returns', node=node,
                )

        self._function_stack[-1].append(node)



In [ ]:
node = astroid.extract_node("return 1")

In [41]:
node_a, node_b = astroid.extract_node("""

def test():

    if True:

        return 5 #@

    return 5 #@

""")

node_a.value
# <Const.int l.4 at 0x7efe621a74e0>

<Const.int l.7 at 0x7f457b709130>

In [38]:

node_a.value.value
# 5

5

In [33]:

node_a.value.value == node_b.value.value
# True

False

In [23]:
node

<Return l.1 at 0x7f457ba69bb0>

In [ ]:
help(node)

In [18]:
node.value

<Const.int l.1 at 0x7f457ba69e50>